In [ ]:
# USAGE
# python train.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPool2D, Dense
from keras.layers import Flatten, Dropout
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping

import keras.backend as K

In [ ]:
scaler = MinMaxScaler()

In [ ]:
def csv_image_generator(inputPath, bs, lb, mode="train", aug=None):
    # open the CSV file for reading
    f = open(inputPath, "r")

    # loop indefinitely
    while True:
        # initialize our batches of images and labels
        images = []
        labels = []

        # keep looping until we reach our batch size
        while len(images) < bs:
            # attempt to read the next line of the CSV file
            line = f.readline()

            # check to see if the line is empty, indicating we have
            # reached the end of the file
            if line == "":
                # reset the file pointer to the beginning of the file
                # and re-read the line
                f.seek(0)
                line = f.readline()

                # if we are evaluating we should now break from our
                # loop to ensure we don't continue to fill up the
                # batch from samples at the beginning of the file
                if mode == "eval":
                    break

            # extract the label and construct the image
            line = line.strip().split(",")
            label = line[0]
            image = np.array([int(x) for x in line[1:]], dtype="uint8")
            image = scaler.fit_transform(image)
            image = image.reshape((1,64, 64, 3))

			# update our corresponding batches lists
            images.append(image)
            labels.append(label)

        ## one-hot encode the labels
        labels = lb.transform(np.array(labels))

        # if the data augmentation object is not None, apply it
        if aug is not None:
            (images, labels) = next(aug.flow(np.array(images),
                labels, batch_size=bs))

        # yield the batch to the calling function
        yield (np.array(images), labels)

In [ ]:

# initialize the paths to our training and testing CSV files
TRAIN_CSV = "../input/flowerdataset/flowerDataset/flowers17_training.csv"
TEST_CSV = "../input/flowerdataset/flowerDataset/flowers17_testing.csv"

# initialize the number of epochs to train for and batch size
NUM_EPOCHS = 75
BS = 32

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = 0
NUM_TEST_IMAGES = 0

# open the training CSV file, then initialize the unique set of class
# labels in the dataset along with the testing labels
f = open(TRAIN_CSV, "r")
labels = set()
testLabels = []

# loop over all rows of the CSV file
for line in f:
	# extract the class label, update the labels list, and increment
	# the total number of training images
	label = line.strip().split(",")[0]
	labels.add(label)
	NUM_TRAIN_IMAGES += 1

# close the training CSV file and open the testing CSV file
f.close()
f = open(TEST_CSV, "r")

# loop over the lines in the testing file
for line in f:
	# extract the class label, update the test labels list, and
	# increment the total number of testing images
	label = line.strip().split(",")[0]
	testLabels.append(label)
	NUM_TEST_IMAGES += 1

# close the testing CSV file
f.close()

# create the label binarizer for one-hot encoding labels, then encode
# the testing labels
lb = LabelBinarizer()
lb.fit(list(labels))
testLabels = lb.transform(testLabels)

In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize both the training and testing image generators
trainGen = csv_image_generator(TRAIN_CSV, BS, lb,
	mode="train", aug=aug)
testGen = csv_image_generator(TEST_CSV, BS, lb,
	mode="train", aug=None)

In [ ]:
def modelCNN(inputShape, classes):    
    inputX = Input(inputShape)

    l1 = Conv2D(32, (3, 3), padding="same", activation='relu')(inputX)
    l1 = BatchNormalization(axis=-1)(l1)
    l1 = Conv2D(32, (3, 3), padding="same", activation='relu')(l1)
    l1 = BatchNormalization(axis=-1)(l1)
    l1 = MaxPool2D(pool_size=(2, 2))(l1)
    l1 = Dropout(0.25)(l1)

    # second CONV => RELU => CONV => RELU => POOL layer set
    l2 = Conv2D(64, (3, 3), padding="same", activation='relu')(l1)
    l2 = BatchNormalization(axis=-1)(l2)
    l2 = Conv2D(64, (3, 3), padding="same", activation='relu')(l2)
    l2 = BatchNormalization(axis=-1)(l2)
    l2 = MaxPool2D(pool_size=(2, 2))(l2)
    l2 = Dropout(0.25)(l2)

    # third CONV => RELU => CONV => RELU => POOL layer set
    l3 = Conv2D(128, (3, 3), padding="same", activation='relu')(l2)
    l3 = BatchNormalization(axis=-1)(l3)
    l3 = Conv2D(128, (3, 3), padding="same", activation='relu')(l3)
    l3 = BatchNormalization(axis=-1)(l3)
    l3 = MaxPool2D(pool_size=(2, 2))(l3)
    l3 = Dropout(0.25)(l3)

    # first (and only) set of FC => RELU layers
    l4 = Flatten()(l3)
    l4 = Dense(512, activation="relu")(l4)
    l4 = Dropout(0.5)(l4)

    # softmax classifier
    predictions = Dense(classes, activation="softmax")(l4)
    
    modelCNN = Model(inputs=inputX, outputs=predictions)
    
    return modelCNN

In [ ]:
K.clear_session()

In [ ]:
model = modelCNN((64,64,3), 17)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=NUM_TRAIN_IMAGES // BS,
	validation_data=testGen,
	validation_steps=NUM_TEST_IMAGES // BS,
	epochs=NUM_EPOCHS)